## IndicTrans2

In [10]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
# from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer
from IndicTransToolkit import IndicProcessor
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm

BATCH_SIZE = 16 # edited from 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
print(DEVICE)

cuda


In [11]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []



    for i in tqdm(range(0, len(input_sentences), BATCH_SIZE)):

    # for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]
        print("batch: ", batch)

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)
        print("batch after pre-process: ", batch)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)
        print("inputs: ", inputs)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1, # TODO temp
            )
            print("generated_tokens: ", generated_tokens.detach().cpu().tolist())

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )
            print("generated_tokens after decode: ", generated_tokens)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

**TODO: CAN I USE SOME CACHE TO STORE THE HF model and tokenizer?**

In [12]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir,  quantization)

ip_en_ind = IndicProcessor(inference=True)

In [14]:
# import the test_sentences_eng.txt file data as sents

sents = ['The river is blue.', 'My uncle wearing blue coat.', 'My maternal aunt went to the river bank.', "The doctor went to bank for money."]

src_lang = "eng_Latn"


tgt_lang = 'hin_Deva'
# print(lang)
translations = batch_translate(sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip_en_ind)


  0%|          | 0/1 [00:00<?, ?it/s]

batch:  ['The river is blue.', 'My uncle wearing blue coat.', 'My maternal aunt went to the river bank.', 'The doctor went to bank for money.']
batch after pre-process:  ['eng_Latn hin_Deva The river is blue .', 'eng_Latn hin_Deva My uncle wearing blue coat .', 'eng_Latn hin_Deva My maternal aunt went to the river bank .', 'eng_Latn hin_Deva The doctor went to bank for money .']
inputs:  {'input_ids': tensor([[    1,     1,     1,     1,     4,    15,    18,  1811,    13,  2225,
             5,     2],
        [    1,     1,     1,     4,    15,   362,  7300,  3559,  2225, 10283,
             5,     2],
        [    4,    15,   362, 14624, 17973,   436,     9,     6,  1811,   978,
             5,     2],
        [    1,     4,    15,    18,   761,   436,     9,   978,    22,   337,
             5,     2]], device='cuda:0'), 'attention_mask': tensor([[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        

/home/sofia/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00,  2.39it/s]

generated_tokens:  [[2, 3228, 52384, 8, 6, 2, 1, 1, 1, 1, 1, 1], [2, 790, 34059, 26, 31523, 4156, 6638, 67, 540, 8, 6, 2], [2, 1653, 8911, 3228, 9, 15933, 167, 3070, 6, 2, 1, 1], [2, 2553, 3688, 9, 44, 1518, 90, 6, 2, 1, 1, 1]]
generated_tokens after decode:  ['नदी नीली है । ', 'मेरे चाचा ने नीला कोट पहना हुआ है । ', 'मेरी नानी नदी के किनारे गई थीं । ', 'डॉक्टर पैसे के लिए बैंक गया । ']


In [1]:
# import random

# x_mean =[]
# n=100000

# # def roll_die():
#     return random.randint(1,6)

# for i in range(n):
#     # roll 10 dice
#     rolls = [roll_die() for _ in range(10)]
#     # calculate the mean
#     x_mean.append(sum(rolls)/10)

# # print(x_mean)
# # plot x mean
# import matplotlib.pyplot as plt
# plt.hist(x_mean, bins=100)
# # show mean in plot
# plt.axvline(x=3.5, color='red')

# plt.show()